In [2]:
from alignment import *
from parsing import parse_matrix_and_gap_cost_in_subst_matrix,read_n_fasta
from msa_sp_score_3k import compute_sp_score
import os

In [3]:
matrix_path = 'input/subst_matrix.txt'
substitution_matrix = parse_matrix_and_gap_cost_in_subst_matrix(matrix_path)

Read and align the sequences from 1-3 to 1-6.

In [7]:
list_of_seq = read_n_fasta('experiments/brca1-testseqs.fasta',6)
results = []
for i in [3,4,5,6]:
        ########## algorithm 
        M = two_approx_algorithm_for_MSA(list_of_seq[:i], substitution_matrix) ## approx 
        alignment_approx_outputfile = 'presentation/aligned_' + str(i)
        approx_alignment = print_alignment(M,alignment_approx_outputfile) ## print alignment to file
    

Compute scores from alignments:

In [9]:
for i in [3,4,5,6]:
    ####### approx
    testfile = 'presentation/aligned_' + str(i)
    if os.path.isfile(testfile):   
        score = compute_sp_score(testfile)
        print("The optimal score is in seqs 1 -",i,":",score)

The optimal score is in seqs 1 - 3 : 792
The optimal score is in seqs 1 - 4 : 2345
The optimal score is in seqs 1 - 5 : 3295
The optimal score is in seqs 1 - 6 : 6005
